In [1]:
import pandas as pd
import torch
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
import os
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
!mkdir "/kaggle/working/labels"
!cp -r "../input/tr-data/images" "/kaggle/working/images"

In [5]:
df = pd.read_csv("../input/annotation-car-train/annotation_car_train_train.csv")
df.head()

In [6]:
df["class"] = 0
df.head()

In [9]:
df

In [11]:
i=346
img = read_image("../input/tr-data/images/"+df["file"][i]+'.jpg')

# bounding box in (xmin, ymin, xmax, ymax) format
# top-left point=(xmin, ymin), bottom-right point = (xmax, ymax)
# bbox = [240, 170, 10, 8]
# bbox = [1989, 1487, 2110, 1547]
bbox = [df['xmin'][i], df["ymin"][i], df['xmax'][i], df["ymax"][i]]
bbox = torch.tensor(bbox, dtype=torch.int)
print(bbox)
print(bbox.size())
bbox = bbox.unsqueeze(0)
print(bbox.size())

# draw bounding box on the input image
img=draw_bounding_boxes(img, bbox, width=3, colors=(255,255,0))

# transform it to PIL image and display
img = torchvision.transforms.ToPILImage()(img)
img

In [7]:
df = df.drop(["name"], axis=1)
df

In [8]:
for i in range(len(df.file)):
    df["file"][i] = df["file"][i]+'.jpg'
df

In [9]:
imgs = os.listdir('/kaggle/working/images')
imgs_train, imgs_val = train_test_split(imgs, test_size=0.05)

In [10]:
import yaml
cwd = '/kaggle/working/'
data = dict(
    train =  cwd + 'train.txt',
    val   =  cwd + 'val.txt',
    nc    = 1,
    names = ['licence'],
)

with open(cwd + 'bgr.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
with open(cwd + 'train.txt', 'w') as f:
    for path in imgs_train:
        f.write(cwd+'images/'+path+'\n')
with open(cwd + 'val.txt', 'w') as f:
    for path in imgs_val:
        f.write(cwd+'images/'+path+'\n')

In [11]:
for file in imgs:
#     print(file)
#     file = file.split('.')[0]
    bboxs = []
#     print(file)
    for _,row in df[df['file'] == file].iterrows():
        bbox = [str(0), str(row['Xcent']), str(row['Ycent']), str(row['boxW']), str(row['boxH'])]
#         print(bbox)
        bbox = ' '.join(bbox)
        bboxs.append(bbox)
    with open(cwd+'labels/'+file.split('.')[0]+'.txt', 'w') as f:
        bboxs = '\n'.join(bboxs)
        f.write(bboxs)

In [12]:
print(bboxs)

In [13]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

from yolov5 import utils
display = utils.notebook_init()

In [ ]:
%%time
!python train.py --img 640\
--batch 10\
--epochs 5\
--data /kaggle/working/bgr.yaml\
--weights /kaggle/input/carmodelupdate/new_car_model_up.pt

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(30,15))
# plt.axis('off')
# plt.imshow(plt.imread('/kaggle/working/yolov5/runs/train/exp/results.png'))

In [ ]:
# fig, ax = plt.subplots(2, 2, figsize = (2*8,2*5), constrained_layout = True)
# for row in range(2):
#     ax[row][0].imshow(plt.imread(f'/kaggle/working/yolov5/runs/train/exp/val_batch{row}_labels.jpg'))
#     ax[row][0].set_xticks([])
#     ax[row][0].set_yticks([])
#     ax[row][0].set_title(f'/kaggle/working/yolov5/runs/train/exp/val_batch{row}_labels.jpg', fontsize = 12)
    
#     ax[row][1].imshow(plt.imread(f'/kaggle/working/yolov5/runs/train/exp/val_batch{row}_pred.jpg'))
#     ax[row][1].set_xticks([])
#     ax[row][1].set_yticks([])
#     ax[row][1].set_title(f'/kaggle/working/yolov5/runs/train/exp/val_batch{row}_pred.jpg', fontsize = 12)
# plt.show()